In [66]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
pd.options.display.float_format = '{:,}'.format

In [68]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
DIR = '/content/gdrive/My Drive/Goodreads/data'
DIR_GENRE = '/content/gdrive/My Drive/Goodreads/data/genre/'

In [0]:
df_books = pd.read_csv(os.path.join(DIR, 'df_books.csv'))

In [71]:
df_books = df_books.sort_values('ratings_count', ascending = False).reset_index(drop=True)
print(df_books.shape)
df_books.head()

(276027, 6)


,average_rating,book_id,image_url,title,description,ratings_count
0,4.34,2767052,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965
1,4.45,3,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497
2,3.57,41865,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381
3,4.26,2657,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518
4,3.89,4671,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812


In [0]:
df_books = df_books.head(10000)

In [0]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer


contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", \
                    "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", \
                    "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", \
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", \
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    text = replace_contractions(text)
    text = text.replace('\n',' ')
    text = text.replace('.','')
    text = text.replace(',','')


    return(text)

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", \
                    "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", \
                    "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", \
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", \
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [74]:
df_books["describ_clean"] = 1
print(len(df_books))

10000


In [75]:
for i in range (len(df_books)):
  df_books["describ_clean"][i] = clean_text(df_books["description"][i])
  if (i % 1000 == 0):
    print(i, end=',\n')
    
#df_reviews["review_clean"] = df_reviews["review_text"].apply(lambda x: clean_text(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0,
1000,
2000,
3000,
4000,
5000,
6000,
7000,
8000,
9000,


In [0]:
export_csv = df_books.to_csv (r'/content/gdrive/My Drive/Goodreads/data/df_books_clean.csv', index = None, header=True)

In [77]:
df_books.head()

,average_rating,book_id,image_url,title,description,ratings_count,describ_clean
0,4.34,2767052,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965,win make famous losing mean certain death the ...
1,4.45,3,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497,harry potter's life miserable parent dead he i...
2,3.57,41865,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381,three thing absolutely positive first edward v...
3,4.26,2657,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518,unforgettable novel childhood sleepy southern ...
4,3.89,4671,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812,great gatsby scott fitzgerald's third book sta...


In [78]:
df_books.describ_clean[2]

'three thing absolutely positive first edward vampire second part him--and know dominant part might be--that thirsted blood and third unconditionally irrevocably love him in first book twilight saga internationally bestselling author stephenie meyer introduces bella swan edward cullen pair star-crossed lover whose forbidden relationship ripen backdrop small-town suspicion mysterious coven vampire love story bite'

In [79]:
#Fit TFIDF 
#Learn vocabulary and tfidf from all style_ids.

import pickle
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer
#from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
#from scipy import sparse
#import re
#import gensim
#from skimage import io
#import matplotlib.pyplot as plt
#from gensim.test.utils import get_tmpfile

tf = TfidfVectorizer(analyzer='word', 
                     min_df=10,
                     ngram_range=(1, 2),
                     #max_features=1000,
                     stop_words='english')
tf.fit(df_books['describ_clean'])

#Transform style_id products to document-term matrix.
tfidf_matrix = tf.transform(df_books['describ_clean'])
#pickle.dump(tf, open("../models/tfidf_model.pkl", "wb"))


    
path_model = '/content/gdrive/My Drive/Goodreads/models/' 
os.makedirs(path_model, exist_ok=True) 
pickle.dump(tf, open(path_model+'tfidf_model.pkl', "wb"))


print(tfidf_matrix.shape)

(10000, 10643)


In [80]:
# Compress with SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
latent_matrix = svd.fit_transform(tfidf_matrix)
pickle.dump(svd, open(path_model+"svd_model.pkl", "wb"))

print(latent_matrix.shape)

(10000, 500)


In [81]:
n = 100 #pick components
#Use elbow and cumulative plot to pick number of components. 
#Need high ammount of variance explained. 
doc_labels = df_books[['book_id','title','image_url']]
svd_feature_matrix = pd.DataFrame(latent_matrix[:,0:n] ,index=doc_labels)
print(svd_feature_matrix.shape)
svd_feature_matrix.head()

pickle.dump(svd_feature_matrix, open(path_model+"lsa_embeddings.pkl", "wb"))

(10000, 100)


Doc2Vec Model



In [0]:
import re
import gensim

#Use reviews for vocabulary 
desrcibes = df_books.describ_clean.values.tolist()

documents = []
for i in range(len(df_books)):
    mystr = desrcibes[i]
    #mystr = mystr + descriptions[i]
    documents.append(re.sub("[^\w]", " ",  mystr).split())

In [92]:
print(len(df_books))
print(len(documents))

10000
10000


In [0]:
import re
import gensim

formatted_documents = [gensim.models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]

model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=5, epochs=200, seed=0, window=3, dm=1)
model.build_vocab(formatted_documents)

In [94]:
len(formatted_documents)

10000

In [95]:
%time model.train(formatted_documents, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 12min 28s, sys: 1min 8s, total: 13min 37s
Wall time: 7min 21s


In [96]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile(path_model+"doc2vec_model")
model.save(path_model+"doc2vec_model")
model = gensim.models.doc2vec.Doc2Vec.load(path_model+"doc2vec_model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
vector = model.infer_vector(doc_words=["this", "is", "a", "test"], epochs=50)
vector

array([ 0.5199108 , -0.9004693 , -0.02126042, -0.5897339 , -0.24350978,
        0.2694342 , -0.49371433, -0.5869183 , -0.2992361 ,  0.19193925,
        0.68869936,  0.7399663 ,  0.12753426,  0.1375813 ,  0.11851804,
        0.7682578 , -0.02046373, -0.5128027 ,  0.02884166, -0.16704261,
        0.03990236,  0.32756996, -0.30781388, -0.06016935,  0.7373465 ,
        0.6942833 ,  0.0377945 ,  1.0808781 ,  0.8538564 , -0.7552733 ,
        0.17444703,  0.47337103,  0.972844  ,  0.04480402, -0.9265533 ,
       -0.43439582, -0.7885536 ,  0.46265408, -0.44659242, -0.19045691,
        0.1914479 ,  0.2591057 ,  0.46054626,  0.3229093 ,  0.10967737,
        0.28422672, -0.0158411 ,  0.1182273 ,  0.214657  , -0.19551022,
       -0.02003466,  0.07047599,  0.07164456, -0.02337483, -0.47757465,
        0.02409443,  0.53115433,  0.40139613, -0.20355903, -0.0241091 ,
       -0.09568987, -0.02034253, -0.17488809,  0.19694462,  0.0265969 ,
       -0.66881806, -0.3146997 , -0.1098014 , -0.6222785 , -0.43

In [98]:
doctovec_feature_matrix = pd.DataFrame(model.docvecs.vectors_docs, index=doc_labels)
print(doctovec_feature_matrix.shape)
pickle.dump(doctovec_feature_matrix, open(path_model+"doctovec_embeddings.pkl", "wb"))
doctovec_feature_matrix.head(3)

(10000, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
"(2767052, The Hunger Games (The Hunger Games, #1), https://images.gr-assets.com/books/1447303603m/2767052.jpg)",2.793867588043213,-1.315008521080017,4.3620076179504395,-0.9533140063285828,-4.481552600860596,-0.5669111013412476,-2.610900402069092,-1.1310807466506958,4.913485050201416,2.388441324234009,-2.6023900508880615,0.13957807421684265,0.27334973216056824,1.9029911756515503,0.9409551620483398,1.821941614151001,-0.14905574917793274,0.008084370754659176,-0.23142217099666595,-2.196979284286499,0.1449321210384369,-0.4380040466785431,-3.1669352054595947,0.9374728798866272,-0.9975630640983582,1.1303919553756714,2.549800395965576,2.7843868732452393,-0.9116271734237671,-2.6061739921569824,0.14716367423534393,-0.9507983326911926,3.7725636959075928,-0.46429482102394104,-3.367699146270752,2.8730428218841553,-0.8531596660614014,1.2549179792404175,1.7085014581680298,-0.44729769229888916,...,2.1471824645996094,1.1745731830596924,0.9395421743392944,0.7576508522033691,3.108999490737915,-2.1049606800079346,2.201058864593506,0.13360294699668884,-3.0435309410095215,1.726237177848816,-0.1324629932641983,-0.7765334844589233,-0.5554754137992859,-0.8108124136924744,0.06660603731870651,0.7096542119979858,1.700522780418396,-0.46108657121658325,-1.617978572845459,-0.6592949628829956,-2.3900020122528076,-2.5998270511627197,3.31221866607666,0.6361701488494873,1.5743762254714966,-0.8582370281219482,2.409395694732666,-1.3299356698989868,0.4738236665725708,-2.495497226715088,-0.4447048306465149,-2.224426031112671,-0.4769255518913269,-0.17273065447807312,2.076918125152588,0.5873788595199585,-0.19725655019283295,-3.071436882019043,-0.7639278173446655,1.2683916091918945
"(3, Harry Potter and the Sorcerer's Stone (Harry Potter, #1), https://images.gr-assets.com/books/1474154022m/3.jpg)",0.3342093825340271,0.49555036425590515,-1.4561134576797485,-3.7611641883850098,1.4656623601913452,0.7177755832672119,-0.1007063016295433,-0.24711714684963226,0.7696372270584106,4.32495641708374,1.4794120788574219,0.4269409775733948,-2.0656018257141113,-1.742795467376709,-1.1004945039749146,0.9052801132202148,0.11419486999511719,-2.500852108001709,-1.3020087480545044,1.3961868286132812,1.8616141080856323,0.35357096791267395,-3.15783953666687,-4.705731391906738,1.868160605430603,0.13495513796806335,1.9389857053756714,2.6407554149627686,0.203363299369812,-1.0851795673370361,-0.7412430644035339,3.255465030670166,2.80594801902771,-1.9327529668807983,-0.9110897779464722,-0.387969434261322,0.6725270748138428,-0.04553753510117531,-1.171799898147583,-1.5608669519424438,...,2.1585195064544678,-0.8643527626991272,-0.7909695506095886,-0.3649592697620392,0.5178773403167725,-1.3233397006988525,-0.04582388699054718,-0.9723125696182251,-1.345476508140564,-0.02617296390235424,3.1249399185180664,2.180628776550293,-0.4975413382053375,-1.1666117906570435,1.9976215362548828,0.3096955716609955,-0.5918468832969666,1.9269095659255981,-1.3890602588653564,0.7133981585502625,-1.543015956878662,-1.7854835987091064,-1.0365111827850342,2.9533770084381104,-1.388226866722107,0.6144031882286072,2.241776466369629,-0.933677077293396,-0.17967338860034943,2.0791523456573486,-2.6901891231536865,1.2632455825805664,-1.0185071229934692,0.6865766048431396,4.627225875854492,3.262214183807373,-1.7467719316482544,0.9394472241401672,-1.5149189233779907,0.2643302381038666
"(41865, Twilight (Twilight, #1), https://images.gr-assets.com/books/1361039443m/41865.jpg)",2.187035322189331,0.647415041923523,0.7907336354255676,0.0051705013029277325,-0.24498578906059265,0.426969975233078,0.9567239284515381,-2.0466599464416504,0.9049897789955139,2.0601234436035156,0.1647837609052658,3.2652180194854736,-0.5975408554077148,0.4420263469219208,1.9528687000274658,0.6984885334968567,-1.9480886459350586,-0.0

In [99]:
doc_labels

,book_id,title,image_url
0,2767052,"The Hunger Games (The Hunger Games, #1)",https://images.gr-assets.com/books/1447303603m...
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...
2,41865,"Twilight (Twilight, #1)",https://images.gr-assets.com/books/1361039443m...
3,2657,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680m...
4,4671,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
...,...,...,...
9995,16185789,Wounded,https://images.gr-assets.com/books/1354323242m...
9996,593236,Runny Babbit: A Billy Sook,https://images.gr-assets.com/books/1421009701m...
9997,290369,Safely Home,https://images.gr-assets.com/books/1406506307m...
9998,22521973,Last Stop on Market Street,https://images.gr-assets.com/books/1453342770m...
